Retrieval Augmented Generation

We create retriever-reader pipelines to provide retrieval augmented systems based on large language models (LLMs) such as OpenAI's ChatGPT(gpt-3.5-turbo), InstructGPT(text-davinci-003), FLAN-T5 and LLMs accessed through the BAM service (IBM Internal).
These systems generate answers conditioned on the retrieved passages.

The OpenAI models are accessed using an API KEY that can be obtained at (https://platform.openai.com/account/api-keys).

In [1]:
from primeqa.components.retriever.dense import ColBERTRetriever
from primeqa.components.reader.prompt import PromptGPTReader
from primeqa.pipelines.qa_pipeline import QAPipeline
import json

{"time":"2023-03-06 11:10:11,177", "name": "faiss.loader", "level": "INFO", "message": "Loading faiss with AVX2 support."}
{"time":"2023-03-06 11:10:11,255", "name": "faiss.loader", "level": "INFO", "message": "Successfully loaded faiss with AVX2 support."}


In [2]:
# setup ColBERT index
index_root = "/dccstor/colbert-ir/franzm/indexes/oct2_10_11/oct2_10_11_exp/indexes/"
index_name = "oct2_10_11_indname"
collection = "/dccstor/avi7/neural_ir/colbert/data/psgs_w100.tsv"


retriever = ColBERTRetriever(index_root = index_root, 
                                     index_name = index_name, 
                                     collection = collection, 
                                     max_num_documents = 3)
retriever.load()

[Mar 06, 11:10:53] #> base_config.py from_path /dccstor/colbert-ir/franzm/indexes/oct2_10_11/oct2_10_11_exp/indexes//oct2_10_11_indname/metadata.json
[Mar 06, 11:10:53] #> base_config.py from_path args loaded! 
[Mar 06, 11:10:53] #> base_config.py from_path args replaced ! 
[Mar 06, 11:11:03] #>>>>> at ColBERT name (model type) : /dccstor/colbert-ir/franzm/experiments/oct2_7_12_1.5e-06/none/2022-10/09/15.21.39/checkpoints/colbert.dnn.batch_91287.model
[Mar 06, 11:11:03] #>>>>> at BaseColBERT name (model type) : /dccstor/colbert-ir/franzm/experiments/oct2_7_12_1.5e-06/none/2022-10/09/15.21.39/checkpoints/colbert.dnn.batch_91287.model
[Mar 06, 11:11:13] factory model type: xlm-roberta-large
[Mar 06, 11:11:27] get query model type: xlm-roberta-large
[Mar 06, 11:11:27] get doc model type: xlm-roberta-large
[Mar 06, 11:11:38] #> Loading codec...
[Mar 06, 11:11:38] #> base_config.py from_path /dccstor/colbert-ir/franzm/indexes/oct2_10_11/oct2_10_11_exp/indexes//oct2_10_11_indname/metadata.js

We provide three examples of loading models to generate answers from the prompt: GPT, Flan-T5-xxl from HuggingFace, and Flan-T5-xxl from BAM services

In [4]:
# setup a Prompt GPT Reader: We support gpt-3.5-turbo and text-davinci-003
reader = PromptGPTReader(model_name='gpt-3.5-turbo', api_key='API KEY HERE')
reader.load()

In [5]:
# setup the pipeline
pipeline = QAPipeline(retriever, reader)

21015325it [01:06, 314724.68it/s]


In [7]:
# start asking questions
questions = ["Does smoking marijuana impair driving ?"]
prompt_prefix = "Answer the following question after looking at the text."
answers = pipeline.run(questions, prefix=prompt_prefix)
print(json.dumps(answers, indent=4))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 54.54it/s]


{
    "0": {
        "answers": {
            "text": "According to the text, consuming marijuana can impair driving ability and is illegal in Colorado if it impairs the ability to drive, even if it is prescribed by a doctor or legally acquired. Studies suggest that marijuana use may increase the risk of fatal crashes, but other studies have found lower or no elevated crash risk estimates. Therefore, the answer is yes, smoking marijuana can impair driving."
        },
        "passages": [
            "\"Driving under the influence\"\n \"driving under the influence (DUI). However, no matter the level of THC, law enforcement officers base arrests on observed impairment.\"\" In Colorado, if consumption of marijuana is impairing your ability to drive, \"\"it is illegal for you to be driving, even if that substance is prescribed [by a doctor] or legally acquired.\"\" Prescription medications such as opioids and benzodiazepines often cause side effects such as excessive drowsiness, and, in 

In [8]:
# start asking questions
questions = ["number of participating countries in tour de france 2017 ?"]
prompt_prefix = "Answer the following question after looking at the text."
answers = pipeline.run(questions, prefix=prompt_prefix)
print(json.dumps(answers, indent=4))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 52.38it/s]


{
    "0": {
        "answers": {
            "text": "How many participating countries were there in the Tour de France 2017?\n\nThere were 32 participating countries in the Tour de France 2017."
        },
        "passages": [
            "\"2017 Tour de France\"\n \"de France. The total number of riders that finished the race was 167. The riders came from 32 countries. Six countries had more than 10 riders in the race: France (39), Italy (18), Belgium (16), Germany (16), the Netherlands (15), and Spain (13). The average age of riders in the race was 29.4 years, ranging from the 22-year-old \u00c9lie Gesbert () to the 40-year-old Haimar Zubeldia (). had the youngest average age while had the oldest. The teams entering the race were: In the lead up to the 2017 Tour de France, Chris Froome () was seen by many pundits\"",
            "\"2018 Tour de France\"\n \"reduced the number of riders per team for Grand Tours from 9 to 8, resulting in a start list total of 176, instead of the usu

In [9]:
questions = ["How many area codes are in New Hampshire ?"]
prompt_prefix = "Answer the following question after looking at the text."
answers = pipeline.run(questions, prefix=prompt_prefix)
print(json.dumps(answers, indent=4))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 60.56it/s]


{
    "0": {
        "answers": {
            "text": "There is only one area code in New Hampshire, which is area code 603."
        },
        "passages": [
            "\"Area code 603\"\n \"previously allocated telephone numbers, as well as number pooling, the exhaustion time frame has been moved beyond 2025. Since New Hampshire has only one area code, callers in the state can reach any other telephone within the 603 area code with 7-digit dialing. Area code 603 Area code 603 is the sole area code for the U.S. state of New Hampshire in the North American Numbering Plan (NANP). It was created as one of the original 86 numbering plan areas in October 1947. As of April 2011, the 603 code was nearing exhaustion and a second area code for New\"",
            "\"Area code 603\"\n \"Area code 603 Area code 603 is the sole area code for the U.S. state of New Hampshire in the North American Numbering Plan (NANP). It was created as one of the original 86 numbering plan areas in October 1947.

In [10]:
questions = ["what engine is in the audi r8 v10"]
prompt_prefix = "Answer the following question after looking at the text."
answers = pipeline.run(questions, prefix=prompt_prefix)
print(json.dumps(answers, indent=4))

100%|███████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 114.92it/s]


{
    "0": {
        "answers": {
            "text": "What engine is in the Audi R8 V10?\n\nThe engine in the Audi R8 V10 is a 5.2 litre FSI engine, based on the unit in the Lamborghini Gallardo LP560-4."
        },
        "passages": [
            "\"Audi R18\"\n \"gearbox is electrically controlled instead of pneumatically controlled, saving weight by eliminating the pneumatic system. Despite the capacity reduction, the 3.7L V6 is claimed to develop more than of power. This is less than the outgoing R15, but the V6 engine's fuel consumption will more than likely be lower than that of the outgoing V10 engine on the R15. The new engine has a single Garrett (Honeywell Turbo Technologies) TR30R VGT turbocharger, as opposed to the twin TR30R configuration of both the Peugeot 908 HDi FAP and the previous Audi R15 TDI. The R18's V6 engine exhausts inwards between the\"",
            "\"Audi RS 6\"\n \"steering wheel movements. The facelifted Audi A6, released as a 2009 model, received sim

In [11]:
#setup a FlanT5PromptReader: note this loads a Flan T5 xxl model
from primeqa.components.reader.prompt import PromptFLANT5Reader
reader = PromptFLANT5Reader(model_name="google/flan-t5-xxl")
reader.load()

In [12]:
# setup the pipeline
pipeline = QAPipeline(retriever, reader)

21015325it [01:03, 328784.64it/s]


In [13]:
# start asking questions
questions = ["Does smoking marijuana impair driving ?"]
prompt_prefix = "Answer the following question after looking at the text."
answers = pipeline.run(questions,prefix=prompt_prefix)
print(json.dumps(answers, indent=4))

100%|█████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 59.25it/s]


{
    "0": {
        "answers": {
            "text": "Smoking marijuana impairs driving. \"\"it is illegal for you to be driving, even if that substance is prescribed [by a doctor] or legally acquired.\"\" Prescription medications such as opioids and benzodiazepines often cause side effects such as excessive drowsiness, and, in the case of opioids, nausea. Other prescription drugs including antiepileptics and antidepressants are now also believed to have the same effect. In the last ten years, there has been an increase\", \"Child development\" \"the child. Parental marijuana exposure may have long-term emotional and behavioral consequences."
        },
        "passages": [
            "\"Driving under the influence\"\n \"driving under the influence (DUI). However, no matter the level of THC, law enforcement officers base arrests on observed impairment.\"\" In Colorado, if consumption of marijuana is impairing your ability to drive, \"\"it is illegal for you to be driving, even if th

In [14]:
questions = ["How many area codes are in New Hampshire ?"]
prompt_prefix = "Answer the following question after looking at the text."
prompt_suffix = "Answer"
answers = pipeline.run(questions, prefix=prompt_prefix, suffix=prompt_suffix)
print(json.dumps(answers, indent=4))

100%|█████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 45.23it/s]


{
    "0": {
        "answers": {
            "text": "3 area codes are in New Hampshire. The original Megabucks, a \"classic\"\" six-number game, began in 1985. Megabucks\"\"\"\" the state's area code is launched on December 15, 2017. These games are always drawn in New Hampshire: Pick 3 and Pick 4 are numbers games played twice daily, including Sundays. Prizes and options vary. The original Megabucks, a \"classic\"\" six-number game, began in 1985."
        },
        "passages": [
            "\"Area code 603\"\n \"previously allocated telephone numbers, as well as number pooling, the exhaustion time frame has been moved beyond 2025. Since New Hampshire has only one area code, callers in the state can reach any other telephone within the 603 area code with 7-digit dialing. Area code 603 Area code 603 is the sole area code for the U.S. state of New Hampshire in the North American Numbering Plan (NANP). It was created as one of the original 86 numbering plan areas in October 1947. As 

In [15]:
#Reader with the bam service (https://bam.res.ibm.com/): note this loads a Flan T5 xxl model
from primeqa.components.reader.prompt import BAMReader
reader = BAMReader(api_key='API_KEY_HERE')
reader.load()

In [16]:
# setup the pipeline
pipeline = QAPipeline(retriever, reader)

21015325it [01:19, 265482.26it/s]


In [17]:
# start asking questions
questions = ["Does smoking marijuana impair driving ?"]
prompt_prefix = "Answer the following question after looking at the text."
prompt_suffix = "Answer"
answers = pipeline.run(questions, prefix=prompt_prefix, suffix=prompt_suffix)
print(json.dumps(answers, indent=4))

100%|█████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.44s/it]


{
    "0": {
        "answers": {
            "text": "no matter the level of THC, law enforcement officers base arrests on observed impairment.\"\" In Colorado, if consumption of marijuana is impairing your ability to drive, \"\"it is illegal for you to be driving, even if that substance is prescribed [by a doctor] or legally acquired.\"\" Prescription medications such as opioids and benzodiazepines often cause side effects such as excessive drowsiness, and, in the case of opioids, nausea. Other prescription drugs including antiepileptics and antidepressants are now also believed to have the same effect. In the last ten years, there has been an increase\", \"Child development\" \"the child. Parental marijuana exposure may have long-term emotional and behavioral consequences. A ten-year-old child who had been exposed to the drug during pregnancy reported more depressive symptoms than fetuses unexposed. Some short-term effects include executive function impairment, reading difficulty, a

In [18]:
# start asking questions
questions = ["number of participating countries in tour de france 2017 ?"]
prompt_prefix = "Answer the following question after looking at the text."
prompt_suffix = "Answer"
answers = pipeline.run(questions, prefix=prompt_prefix, suffix=prompt_suffix)
print(json.dumps(answers, indent=4))

100%|█████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 52.06it/s]


{
    "0": {
        "answers": {
            "text": "32 countries. Six countries had more than 10 riders in the race: France (39), Italy (18), Belgium (16), Germany (16), the Netherlands (15), and Spain (13). The average age of riders in the race was 29.4 years, ranging from the 22-year-old \u00c9lie Gesbert () to the 40-year-old Haimar Zubeldia (). had the youngest average age while had the oldest. The teams entering the race were: In the lead up to the 2017 Tour de France, Chris Froome () was seen by many pundits"
        },
        "passages": [
            "\"2017 Tour de France\"\n \"de France. The total number of riders that finished the race was 167. The riders came from 32 countries. Six countries had more than 10 riders in the race: France (39), Italy (18), Belgium (16), Germany (16), the Netherlands (15), and Spain (13). The average age of riders in the race was 29.4 years, ranging from the 22-year-old \u00c9lie Gesbert () to the 40-year-old Haimar Zubeldia (). had the youn